## json 파일 생성

In [8]:
import os
import json
import numpy as np
import cv2
from skimage.measure import find_contours, approximate_polygon
import glob

# 이미지 폴더 경로
image_folder_path = '/Users/nayoungmin/Desktop/train_image'

# JSON 파일을 위한 기본 구조
coco_format = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "red", "supercategory": ""},
        {"id": 2, "name": "green", "supercategory": ""}
    ]
}

# 고유 ID 생성을 위한 카운터
annotation_id = 1
image_id = 1

# 이미지 경로 가져오기
image_paths = glob.glob(os.path.join(image_folder_path, '*.png'))

for img_path in image_paths:
    # 이미지 로드
    image = cv2.imread(img_path)
    height, width, _ = image.shape
    
    # 이미지 파일명 추출
    image_filename, _ = os.path.splitext(os.path.basename(img_path))
    
    # 이미지 정보 추가
    coco_format["images"].append({
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": image_filename + '.png'  # JSON 파일에서도 확장자를 포함하도록 수정
    })

    # 색상별로 마스크 생성
    red_channel, green_channel, blue_channel = cv2.split(image)
    red_mask = (red_channel > green_channel) & (red_channel > blue_channel)
    green_mask = (green_channel > red_channel) & (green_channel > blue_channel)

    # 각 마스크에 대해 contours 추출
    for mask, category_id in [(red_mask, 1), (green_mask, 2)]:
        contours = find_contours(mask.astype(np.uint8), 0.5)
        
        for contour in contours:
            # COCO 형식에 맞게 다각형 근사화 및 어노테이션 정보 생성
            poly = approximate_polygon(contour, tolerance=2.5)
            segmentation = poly.ravel().tolist()
            
            # 어노테이션 정보 추가
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "segmentation": [segmentation],
                "area": int(np.sum(mask)),  # 가장 단순한 영역 계산
                "bbox": list(cv2.boundingRect(contour.astype(np.float32))),
                "iscrowd": 0
            })
            annotation_id += 1
    
    image_id += 1

# 모든 이미지에 대한 어노테이션을 하나의 JSON 파일로 저장
json_save_path = os.path.join('/Users/nayoungmin/Desktop/train_info/', 'annotations.json')
with open(json_save_path, 'w') as f:
    json.dump(coco_format, f, indent=4)


In [9]:
import os
import json
import numpy as np
import cv2
from skimage.measure import find_contours, approximate_polygon
import glob

# 이미지 폴더 경로
image_folder_path = '/Users/nayoungmin/Desktop/valid_image'

# JSON 파일을 위한 기본 구조
coco_format = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "red", "supercategory": ""},
        {"id": 2, "name": "green", "supercategory": ""}
    ]
}

# 고유 ID 생성을 위한 카운터
annotation_id = 1
image_id = 1

# 이미지 경로 가져오기
image_paths = glob.glob(os.path.join(image_folder_path, '*.png'))

for img_path in image_paths:
    # 이미지 로드
    image = cv2.imread(img_path)
    height, width, _ = image.shape
    
    # 이미지 파일명 추출
    image_filename, _ = os.path.splitext(os.path.basename(img_path))
    
    # 이미지 정보 추가
    coco_format["images"].append({
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": image_filename + '.png'  # JSON 파일에서도 확장자를 포함하도록 수정
    })

    # 색상별로 마스크 생성
    red_channel, green_channel, blue_channel = cv2.split(image)
    red_mask = (red_channel > green_channel) & (red_channel > blue_channel)
    green_mask = (green_channel > red_channel) & (green_channel > blue_channel)

    # 각 마스크에 대해 contours 추출
    for mask, category_id in [(red_mask, 1), (green_mask, 2)]:
        contours = find_contours(mask.astype(np.uint8), 0.5)
        
        for contour in contours:
            # COCO 형식에 맞게 다각형 근사화 및 어노테이션 정보 생성
            poly = approximate_polygon(contour, tolerance=2.5)
            segmentation = poly.ravel().tolist()
            
            # 어노테이션 정보 추가
            coco_format["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "segmentation": [segmentation],
                "area": int(np.sum(mask)),  # 가장 단순한 영역 계산
                "bbox": list(cv2.boundingRect(contour.astype(np.float32))),
                "iscrowd": 0
            })
            annotation_id += 1
    
    image_id += 1

# 모든 이미지에 대한 어노테이션을 하나의 JSON 파일로 저장
json_save_path = os.path.join('/Users/nayoungmin/Desktop/valid_info/', 'annotations.json')
with open(json_save_path, 'w') as f:
    json.dump(coco_format, f, indent=4)


In [31]:
from PIL import Image
import numpy as np
import json
from skimage.measure import find_contours

# Function to calculate the bounding box of a binary mask
def mask_to_bbox(mask):
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return [int(cmin), int(rmin), int(cmax), int(rmax)]

# Function to convert mask to polygons
def mask_to_polygon(mask):
    contours = find_contours(mask, 0.5)
    polygons = []
    for contour in contours:
        # Simplify the contour to avoid too many points
        contour = np.flip(contour, axis=1)
        contour = np.around(contour)
        contour = contour.astype(int)
        polygons.append(contour.ravel().tolist())
    return polygons

# Load images from file paths
image_paths = ['/Users/nayoungmin/Desktop/train_image/스크린샷 2024-04-28 오후 8.06.11.png', 
               '/Users/nayoungmin/Desktop/train_image/스크린샷 2024-04-28 오후 8.06.21.png']
images = [Image.open(path) for path in image_paths]

# Convert images to numpy arrays
arrays = [np.array(image) for image in images]

# Define colors in RGB
colors = {
    'Green': [0, 128, 0, 255],
    'Red': [255, 0, 0, 255]
}

# Function to get the binary mask of the given color
def get_mask(array, color):
    return np.all(array == color, axis=-1)

# Function to create an annotation for a single object
def create_annotation(category_id, bbox, image_id, segmentation):
    return {
        "segmentation": segmentation,
        "area": np.sum(np.array(segmentation) != 0),  # Sum of non-zero pixels in the mask
        "iscrowd": 0,
        "image_id": image_id,
        "bbox": bbox,
        "category_id": category_id,
        "id": image_id * 10 + category_id  # Simple formula to generate a unique ID for each annotation
    }

# Initialize COCO-like annotation structure
annotations = {
    "images": [],
    "annotations": [],
    "categories": [
        {"id": 1, "name": "Green"},
        {"id": 2, "name": "Red"}
    ]
}

# Process each image to create annotations
for i, array in enumerate(arrays):
    # Add image info
    annotations['images'].append({
        "id": i,
        "width": array.shape[1],
        "height": array.shape[0],
        "file_name": image_paths[i].split('/')[-1]
    })
    
    # Process each color
    for color_name, color_value in colors.items():
        mask = get_mask(array, color_value)
        if mask.any():
            bbox = mask_to_bbox(mask)
            segmentation = mask_to_polygon(mask)
            annotations['annotations'].append(create_annotation(
                category_id=list(colors.keys()).index(color_name) + 1,
                bbox=bbox,
                image_id=i,
                segmentation=segmentation
            ))

# Save the annotations as a JSON file
annotation_file_path = '/Users/nayoungmin/Desktop/annotations.json'
with open(annotation_file_path, 'w') as file:
    json.dump(annotations, file, indent=4)


In [28]:
from PIL import Image

def convert_to_binary(pixel):
    # 픽셀이 정수일 경우에는 해당 값이 0이 아니면 255로, 그 외에는 0으로 설정
    if isinstance(pixel, int):
        return 255 if pixel != 0 else 0
    else:
        # R 값이 0이 아니면 255로, 그 외에는 0으로 설정
        r = 255 if pixel[0] != 0 else 0
        # G 값이 0이 아니면 255로, 그 외에는 0으로 설정
        g = 255 if pixel[1] != 0 else 0
        # B 값을 0으로 설정
        b = 0
        return (r, g, b)

def process_image(image_path, output_path):
    # 이미지 열기
    image = Image.open(image_path)
    
    # 이미지의 모든 픽셀에 대해 새로운 값을 할당
    image = image.point(convert_to_binary)
    
    # 변경된 이미지 저장
    image.save(output_path)
    print(f"이미지가 성공적으로 변환되어 '{output_path}'에 저장되었습니다.")

# 이미지 경로 지정
input_image_path = "/Users/nayoungmin/Desktop/스크린샷 2024-03-13 오후 10.54.07.png"

# 변환된 이미지 저장 경로 및 파일명 지정
output_image_path = "/Users/nayoungmin/Desktop/converted_image.png"

# 이미지 처리 및 저장
process_image(input_image_path, output_image_path)


이미지가 성공적으로 변환되어 '/Users/nayoungmin/Desktop/converted_image.png'에 저장되었습니다.
